In [1]:
import pandas as pd 
import json
import os 
import pickle
import numpy as np
import re
from collections import Counter

import spacy
from spacy import displacy
nlp = spacy.load("en_core_web_lg")
from spacy.matcher import Matcher
import textacy
from textacy import extract
from textacy import preprocessing


import matplotlib.pyplot as plt
import seaborn as sns
#sns.set_style('whitegrid')
%matplotlib inline

/Users/dea/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv('./pg_catalog.csv')

/Users/dea/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df_raw = df

In [4]:
df = df.dropna(subset=['Authors', 'Title', 'Subjects'])

### Author born and died years
Extract the range, birth and death years of the first author. We will only consider books which have an author and they have years. We will use these as a proxy for years when their books were written, as we don't have this metadata. 

In [5]:
def extract_years(x):
    years = re.findall(r'\d+(?:-\d+)+', x)
    if len(years) > 0:
        return years[0]
    else:
        return 'no year'

In [6]:
def extract_year_born(x):
    if x == 'no year':
        return '9999'
    else:
        return x.split('-')[0]

In [7]:
def extract_year_died(x):
    if x == 'no year':
        return '9999'
    else:
        return x.split('-')[1]

In [8]:
# Add columns with the range, birth and death years
df['first_author_years_range'] = df['Authors'].apply(lambda x : extract_years(x))
df['first_author_born'] = df['first_author_years_range'].apply(lambda x : extract_year_born(x)).astype(int)
df['first_author_died'] = df['first_author_years_range'].apply(lambda x : extract_year_died(x)).astype(int)

/var/folders/q3/7wmphwj9575b1g06mj1hl7gr0000gn/T/ipykernel_50625/2676397994.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['first_author_years_range'] = df['Authors'].apply(lambda x : extract_years(x))
/var/folders/q3/7wmphwj9575b1g06mj1hl7gr0000gn/T/ipykernel_50625/2676397994.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['first_author_born'] = df['first_author_years_range'].apply(lambda x : extract_year_born(x)).astype(int)
/var/folders/q3/7wmphwj9575b1g06mj1hl7gr0000gn/T/ipykernel_50625/2

### Filtering the data

In [9]:
# Remove the authors for whom we don't have birth years 
df = df[df['first_author_born'] != 9999]

In [10]:
# Only need books that have science fiction in the list of subjects
df = df[df['Subjects'].str.contains('Science fiction')]

In [11]:
# Only need 'text' books
df = df[df['Type'] == 'Text']

In [12]:
# Only english books
df = df[df['Language'] == 'en']

In [13]:
# Take authors born within such a range that they reasonably wrote their books about 100 years ago
df = df[(df['first_author_born'] > 1850) & (df['first_author_born'] < 1940)]

## Get the texts

In [14]:
# List of the texts ids that we want 
text_ids = df['Text#'].tolist()

In [15]:
len(text_ids)

2316

In [16]:
missing_books = []
def get_texts(list_ids):
    texts = {}
    for book_id in list_ids:
        # try: 
        #     with open('./gutenberg_clean/{}.txt'.format(book_id)) as f:
        #         text = f.read()
        #         texts.append(text)
        # except:
        #     missing_books.append(book_id)
        try: 
            with open('./gutenberg_clean/{}.txt'.format(book_id)) as f:
                text = f.read()
                texts[book_id] = text
        except:
            try: 
                with open('./gutenberg_clean/pg{}.txt'.format(book_id)) as f:
                    text = f.read()
                    texts[book_id] = text
            except:
                try: 
                    with open('./gutenberg_clean/{}-0.txt'.format(book_id)) as f:
                        text = f.read()
                        texts[book_id] = text
                except:
                    missing_books.append(book_id)
    return texts

In [17]:
texts = get_texts(text_ids)

In [18]:
len(texts)

2316

In [19]:
len(missing_books)

0

In [20]:
missing_books

[]

Creating lists of missing books that can then be scraped

# Extract time-related words

In [21]:
# Find all the texts that mention a century
count_century = 0
texts_with_century = []
for text in texts.values():
    if "century" in text:
        texts_with_century.append(text)
        count_century += 1

In [22]:
count_century

651

In [23]:
matcher = Matcher(nlp.vocab)

In [24]:
def find_pattern(doc):
    matches = matcher(doc) 
    matches_text = []
    for match_id, start, end in matches:
        try: 
            string_id = nlp.vocab.strings[match_id]  # Get string representation
            span = doc[start:end]  # The matched span
            #print(match_id, string_id, start, end, span.text)
            if any(char.isdigit() for char in span.text): 
                #print(span.text)
                matches_text.append(span.text)
        except: 
            continue
    return matches_text

In [25]:
pattern1 = [
    {"LENGTH": {">=": 1}}, {"LENGTH": {">=": 1}}, {"ENT_TYPE": "DATE"}, {"LENGTH": {">=": 1}}, {"LENGTH": {">=": 1}}
]
matcher.add("Century", [pattern1])

In [26]:
# Find all the dete-related noun phrases
matches_date_words = []
count = 0
for book_id, text in texts.items():  
    # try: 
    doc = nlp(text[1 : 1000000]) # because of restriction of spacy
    matches_text = find_pattern(doc)
    matches_date_words.append({ "book_id": book_id, "matches_text": matches_text })
    # except:
    #     continue
    count +=1 
    if (count %100 == 0):
        print(count)
        with open('matches_date_words_{}.json'.format(count), 'w') as f:
            json.dump(matches_date_words, f)
        # with open('matches_date_words_{}.pickle'.format(count), 'wb') as handle:
        #     pickle.dump(matches_date_words, handle, protocol=pickle.HIGHEST_PROTOCOL)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300


In [27]:
# save with json
with open('matches_date_words.json', 'w') as f:
    json.dump(matches_date_words, f)

In [28]:
# load with json 
with open('matches_date_words.json', 'r') as f:
    matches_date_words = json.load(f)

In [29]:
# Filter out to just titles with number over 2000 (or another number in them) 
def get_numbers_above(mystr, min_num=2000):
    nums_greater_than_min_num = []
    if any(char.isdigit() for char in mystr):
        all_nums = (re.findall('\d+', mystr))
        for num in all_nums:
            if int(num) >= min_num:
                nums_greater_than_min_num.append(int(num))
    if len(nums_greater_than_min_num) > 0:
        return mystr, nums_greater_than_min_num
    else:
        return '', ''
        

In [30]:
test = 'hello world 400 2000'
get_numbers_above(test)[0]

'hello world 400 2000'

In [31]:
matches_date_words[0]

{'book_id': 35, 'matches_text': []}

In [32]:
matches_date_words_with_over2000 = {}
for book in matches_date_words:
    matches_text = book['matches_text']
    if len(matches_text):
        valid_matches = []
        valid_years = []
        for entry in matches_text:
            nums_greater_than_min_num = get_numbers_above(entry)[0]
            if len(nums_greater_than_min_num) > 0:
                valid_matches.append(nums_greater_than_min_num)
                valid_years.extend(get_numbers_above(entry)[1])
        if (len(valid_matches) > 0):
            matches_date_words_with_over2000[str(book['book_id'])] = list(set(valid_years))

In [33]:
len(matches_date_words_with_over2000)

263

In [34]:
matches_date_words_with_over2000['18520']

[2354]

In [35]:
# get a lookup of all the resulting titles 
df_filtered = df[df['Text#'].isin(list(matches_date_words_with_over2000.keys()))]

In [36]:
df_filtered.to_csv('scifi_after_2000_2.csv')

Get longer surrounding text around the years for those books that contain years above 2000

In [38]:
def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub) # use start += 1 to find overlapping matches

In [39]:
matches_date_words_2000 = []
count = 0
for book_id, text in texts.items(): 
    if str(book_id) in list(matches_date_words_with_over2000.keys()):
        #print(book_id)
        for year in matches_date_words_with_over2000[str(book_id)]:
            year_locations = list(find_all(text, str(year)))
            if (len(year_locations)> 1):
                print(book_id)
            samples = []
            for loc in year_locations:
                sample = text[loc-200:loc+200]
                samples.append(sample)
        matches_date_words_2000.append({ "book_id": book_id, "matches_text": samples })
        count +=1 

1607
18458
18520
19471
19709
19726
20121
20121
20121
20154
20707
20727
20859
20988
21092
21638
21970
22073
23669
24054
24135
24247
24949
25776
26867
26867
26867
26867
26867
26867
27365
27365
27462
27730
29135
29303
29455
29720
29882
30452
30532
30691
30869
31223
31223
31223
31327
31357
32154
32154
32473
32473
32485
32530
32853
32903
32903
33386
33850
38287
39572
49531
49531
49638
50682
51379
51379
51379
51868
53456
53932
54096
54096
54096
54629
58784
59376
59376
59376
59588
59752
59752
59752
59752
59752
59752
59752
60393
60944
61551
61694
61756
63613
63656
63716
63826
64314
64673
64673
64710
65218
65324
65370
65370
65977
66163
66206
66259
66312
67122
68067
68349
69009
69293
69338
69338
69338
69338
69338
69338
69338
69338
69584
69709
69709


In [63]:
matches_date_words_2000[21]

{'book_id': 19145,
 'matches_text': ['in forces with a space man from Terra to outwit resurgent nonhuman\nAliens. A sequel to _The Stars Are Ours!_ $2.75\n\n\nTHE STARS ARE OURS!\n\n_by Andre Norton_\n\nTo escape the tyranny on Terra in the year 2500, a group of scientists\nmake a last-minute getaway under fire and take off for another planet in\nanother solar system. Their adventures make top-flight entertainment for\nall science-fiction fans. ']}